In [153]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import linear_model
from scipy.sparse import csr_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import os

In [85]:
os.chdir(r'C:\Users\sjayakannan\Documents\AV\CuisineKaggle')
df = pd.read_json(r"train.json")
testset = pd.read_json(r"test.json")

In [73]:
df.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [74]:
testset.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [75]:
df.isnull().sum()

id             0
cuisine        0
ingredients    0
dtype: int64

In [76]:
testset.isnull().sum()

id             0
ingredients    0
dtype: int64

In [48]:
df.cuisine.unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

In [49]:
df.ingredients[0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [50]:
testset.ingredients[0]

['baking powder',
 'eggs',
 'all-purpose flour',
 'raisins',
 'milk',
 'white sugar']

In [87]:
df.ingredients = df.ingredients.str.replace("["," ")
df.ingredients = df.ingredients.str.replace("]"," ")
df.ingredients = df.ingredients.str.replace("'"," ")
df.ingredients = df.ingredients.str.replace(","," ")

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           39774 non-null  int64 
 1   cuisine      39774 non-null  object
 2   ingredients  39774 non-null  object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB


In [86]:
df.ingredients = df.ingredients.astype('str')

In [64]:
df.ingredients[0]

'  romaine lettuce    black olives    grape tomatoes    garlic    pepper    purple onion    seasoning    garbanzo beans    feta cheese crumbles  '

In [88]:
testset.ingredients = testset.ingredients.astype('str')

In [89]:
testset.ingredients = testset.ingredients.str.replace("["," ")
testset.ingredients = testset.ingredients.str.replace("]"," ")
testset.ingredients = testset.ingredients.str.replace("'"," ")
testset.ingredients = testset.ingredients.str.replace(","," ")

In [81]:
testset.ingredients[0]

'  baking powder    eggs    all-purpose flour    raisins    milk    white sugar  '

In [90]:
df.ingredients = df.ingredients.str.lower()
testset.ingredients = testset.ingredients.str.lower()

In [100]:
df.ingredients = df.ingredients.apply(lambda x: word_tokenize(x))
testset.ingredients = testset.ingredients.apply(lambda x: word_tokenize(x))

In [101]:
lemmatizer = WordNetLemmatizer()

In [102]:
def lemmat(wor):
    l = []
    for i in wor:
        l.append(lemmatizer.lemmatize(i))
    return l

In [103]:
df.ingredients = df.ingredients.apply(lemmat)
testset.ingredients = testset.ingredients.apply(lemmat)

In [95]:
import nltk

In [96]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sjayakannan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [99]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sjayakannan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [104]:
df.ingredients[0]#Olives changed to olive

['romaine',
 'lettuce',
 'black',
 'olive',
 'grape',
 'tomato',
 'garlic',
 'pepper',
 'purple',
 'onion',
 'seasoning',
 'garbanzo',
 'bean',
 'feta',
 'cheese',
 'crumbles']

In [105]:
testset.ingredients[0]

['baking',
 'powder',
 'egg',
 'all-purpose',
 'flour',
 'raisin',
 'milk',
 'white',
 'sugar']

In [107]:
type(df.ingredients[0])

list

In [108]:
df.ingredients = df.ingredients.astype('str')
df.ingredients = df.ingredients.str.replace("["," ")
df.ingredients = df.ingredients.str.replace("]"," ")
df.ingredients = df.ingredients.str.replace("'"," ")
df.ingredients = df.ingredients.str.replace(","," ")

In [109]:
testset.ingredients = testset.ingredients.astype('str')
testset.ingredients = testset.ingredients.str.replace("["," ")
testset.ingredients = testset.ingredients.str.replace("]"," ")
testset.ingredients = testset.ingredients.str.replace("'"," ")
testset.ingredients = testset.ingredients.str.replace(","," ")


In [111]:
type(df.ingredients[0])

str

In [112]:
df.ingredients[0]

'  romaine    lettuce    black    olive    grape    tomato    garlic    pepper    purple    onion    seasoning    garbanzo    bean    feta    cheese    crumbles  '

In [113]:
vect = CountVectorizer()

In [114]:
features = vect.fit_transform(df.ingredients)
features

<39774x2788 sparse matrix of type '<class 'numpy.int64'>'
	with 756011 stored elements in Compressed Sparse Row format>

In [115]:
vect.get_feature_names()[1650:1670]

['muffin',
 'mulato',
 'mullet',
 'multi',
 'multigrain',
 'mung',
 'muscadet',
 'muscadine',
 'muscat',
 'muscavado',
 'muscovado',
 'muscovy',
 'mushroom',
 'mussel',
 'mustard',
 'mutton',
 'myzithra',
 'naan',
 'nacho',
 'nakano']

In [117]:
testfeatures = vect.transform(testset.ingredients)
testfeatures

<9944x2788 sparse matrix of type '<class 'numpy.int64'>'
	with 189396 stored elements in Compressed Sparse Row format>

In [118]:
labels = df.cuisine

In [119]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [120]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31819, 2788) (7955, 2788) (31819,) (7955,)


###### LOGISTIC REGRESSION

In [121]:
logreg = LogisticRegression(C=12)
logreg.fit(X_train,y_train)

C:\Users\sjayakannan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=12)

In [122]:
print("Logistic Regression accuracy",logreg.score(X_test, y_test))

Logistic Regression accuracy 0.7749842866121935


In [123]:
logreg.predict(X_test)

array(['southern_us', 'british', 'mexican', ..., 'italian', 'southern_us',
       'moroccan'], dtype=object)

##### SGD Classifier

In [126]:
linearsvm = LinearSVC(random_state=0, max_iter = 1500)
linearsvm.fit(X_train, y_train)

C:\Users\sjayakannan\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(max_iter=1500, random_state=0)

In [127]:
print("Linear SVM accuracy", linearsvm.score(X_test, y_test))

Linear SVM accuracy 0.77950974230044


#### SGD Classifier performs better than Logistic regression

##### Decision Tree

In [145]:
dt = DecisionTreeClassifier(criterion = 'entropy')

In [146]:
dt.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [148]:
y_pred_dt = dt.predict(X_test)

In [159]:
score =accuracy_score(y_test, y_pred_dt)

In [160]:
print("Decision Tree accuracy", score)

Decision Tree accuracy 0.5987429289754871


##### Neural Networks

In [128]:
labelsNN = df.cuisine
labelsNN = pd.get_dummies(labelsNN)
labelsNN = labelsNN.values
labelsNN[0]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [130]:
sparse_dataset = csr_matrix(features)
featuresNN = sparse_dataset.todense()

In [131]:
featuresNN[0]

matrix([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [132]:
X_trainNN, X_testNN, y_trainNN, y_testNN = train_test_split(featuresNN, labelsNN, test_size=0.2)

In [134]:
print(X_trainNN.shape, X_testNN.shape, y_trainNN.shape, y_testNN.shape)

(31819, 2788) (7955, 2788) (31819, 20) (7955, 20)


#### KERAS

In [135]:
import keras
from keras.layers import *

In [138]:
model = keras.models.Sequential()
model.add(Dense(300,input_dim = 2788,activation = 'relu'))
model.add(Dense(500,activation = 'relu'))
model.add(Dense(400,activation = 'relu'))
model.add(Dense(20,activation='softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['categorical_accuracy'])
model.fit(X_trainNN,y_trainNN,epochs=5,shuffle=True, verbose =2,batch_size=500)

Epoch 1/5
64/64 - 1s - loss: 1.4999 - categorical_accuracy: 0.5836
Epoch 2/5
64/64 - 1s - loss: 0.7539 - categorical_accuracy: 0.7783
Epoch 3/5
64/64 - 1s - loss: 0.5699 - categorical_accuracy: 0.8292
Epoch 4/5
64/64 - 1s - loss: 0.4527 - categorical_accuracy: 0.8623
Epoch 5/5
64/64 - 1s - loss: 0.3641 - categorical_accuracy: 0.8899


In [139]:
print("Accuracy with KERAS" ,model.evaluate(X_testNN,y_testNN)[1])

249/249 [==============================] - 0s 2ms/step - loss: 0.7504 - categorical_accuracy: 0.7893
Accuracy with KERAS 0.7893149256706238


#### Keras is giving 78.9 beating SGD classifier. 